In [ ]:
import pandas as pd 
import os
import sys
from sklearn.ensemble import RandomForestClassifier

    ~Datasetimiz Multiclass bolgani uchun Linear Modellardan foydalanilmadi.
    ~ Tree based modellardan foydalanilgani uchun encoded datasetdan foydalanildi


In [8]:
data_path = r'C:\Fatigue_Level\Data\Preprocessed_Data\Encoded Data\human_decision_fatigue_encoded.csv'

df = pd.read_csv(data_path)

df.head()

,Hours_Awake,Decisions_Made,Task_Switches,Avg_Decision_Time_sec,Sleep_Hours_Last_Night,Time_of_Day,Caffeine_Intake_Cups,Stress_Level_1_10,Error_Rate,Cognitive_Load_Score,Fatigue_Level
0,7,28,7,2.30,5.8,1,0,2.4,0.000,2.6,0.0
1,15,77,22,3.65,4.5,0,3,1.9,0.143,4.5,2.0
2,11,57,23,3.67,6.8,3,2,2.1,0.000,4.1,1.0
3,8,39,10,2.39,5.3,0,1,1.0,0.000,2.3,0.0
4,7,46,16,3.05,8.2,3,1,2.8,0.000,3.9,0.0


In [9]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Hours_Awake             25000 non-null  int64  
 1   Decisions_Made          25000 non-null  int64  
 2   Task_Switches           25000 non-null  int64  
 3   Avg_Decision_Time_sec   25000 non-null  float64
 4   Sleep_Hours_Last_Night  25000 non-null  float64
 5   Time_of_Day             25000 non-null  int64  
 6   Caffeine_Intake_Cups    25000 non-null  int64  
 7   Stress_Level_1_10       25000 non-null  float64
 8   Error_Rate              25000 non-null  float64
 9   Cognitive_Load_Score    25000 non-null  float64
 10  Fatigue_Level           25000 non-null  float64
dtypes: float64(6), int64(5)
memory usage: 2.1 MB


In [10]:
df.isnull().sum()

Hours_Awake               0
Decisions_Made            0
Task_Switches             0
Avg_Decision_Time_sec     0
Sleep_Hours_Last_Night    0
Time_of_Day               0
Caffeine_Intake_Cups      0
Stress_Level_1_10         0
Error_Rate                0
Cognitive_Load_Score      0
Fatigue_Level             0
dtype: int64

In [11]:
df.Fatigue_Level.value_counts()

Fatigue_Level
0.0    11729
2.0     8404
1.0     4867
Name: count, dtype: int64

In [12]:
df.columns.to_list()

['Hours_Awake',
 'Decisions_Made',
 'Task_Switches',
 'Avg_Decision_Time_sec',
 'Sleep_Hours_Last_Night',
 'Time_of_Day',
 'Caffeine_Intake_Cups',
 'Stress_Level_1_10',
 'Error_Rate',
 'Cognitive_Load_Score',
 'Fatigue_Level']

# Feature Engineering

In [14]:
# Datani yoqotmaslik uchun kopiya qip olindi
data = df.copy()

In [15]:
data['Sleep_Debt'] = 8 - data['Sleep_Hours_Last_Night']  # Ideal uyqu vaqti 8 soat dep faraz qilindi

In [16]:
# Sleep and Caffeine interraction
data['Sleep_Caffeine_Interaction'] = data['Sleep_Hours_Last_Night'] * data['Caffeine_Intake_Cups']

In [17]:
# Strees and Load Interraction
data['Stress_Cognitive_Interaction'] = data['Stress_Level_1_10'] * data['Cognitive_Load_Score']

In [18]:
# Time per task to compete it
data['Task_Completion_Time'] = data['Decisions_Made'] * data['Avg_Decision_Time_sec']

# Feature Selection for tree based models

In [20]:
from sklearn.model_selection import train_test_split

# In order to do feature selection we need to split the data into train and test sets
X = data.drop(columns= 'Fatigue_Level')
y = data['Fatigue_Level']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [22]:
X_train.info()

<class 'pandas.DataFrame'>
Index: 17500 entries, 4913 to 23654
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Hours_Awake                   17500 non-null  int64  
 1   Decisions_Made                17500 non-null  int64  
 2   Task_Switches                 17500 non-null  int64  
 3   Avg_Decision_Time_sec         17500 non-null  float64
 4   Sleep_Hours_Last_Night        17500 non-null  float64
 5   Time_of_Day                   17500 non-null  int64  
 6   Caffeine_Intake_Cups          17500 non-null  int64  
 7   Stress_Level_1_10             17500 non-null  float64
 8   Error_Rate                    17500 non-null  float64
 9   Cognitive_Load_Score          17500 non-null  float64
 10  Sleep_Debt                    17500 non-null  float64
 11  Sleep_Caffeine_Interaction    17500 non-null  float64
 12  Stress_Cognitive_Interaction  17500 non-null  float64
 13  Task_Completio

In [23]:
X_test.info()

<class 'pandas.DataFrame'>
Index: 7500 entries, 6868 to 4731
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Hours_Awake                   7500 non-null   int64  
 1   Decisions_Made                7500 non-null   int64  
 2   Task_Switches                 7500 non-null   int64  
 3   Avg_Decision_Time_sec         7500 non-null   float64
 4   Sleep_Hours_Last_Night        7500 non-null   float64
 5   Time_of_Day                   7500 non-null   int64  
 6   Caffeine_Intake_Cups          7500 non-null   int64  
 7   Stress_Level_1_10             7500 non-null   float64
 8   Error_Rate                    7500 non-null   float64
 9   Cognitive_Load_Score          7500 non-null   float64
 10  Sleep_Debt                    7500 non-null   float64
 11  Sleep_Caffeine_Interaction    7500 non-null   float64
 12  Stress_Cognitive_Interaction  7500 non-null   float64
 13  Task_Completion_

For classifications, we need target variabels to be integer not float

In [25]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [26]:
y_train

4913     2
9338     1
24211    2
18791    0
16066    0
        ..
21575    0
5390     2
860      0
15795    2
23654    2
Name: Fatigue_Level, Length: 17500, dtype: int64

# Feature Selection

In [53]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Muhim featurelarni aniqlash
feature_importances = model.feature_importances_

# Muhimlilik darajasi boyicha sortirofka qilib olish
important_features = pd.Series(feature_importances, index=X_train.columns).sort_values(ascending=False)
print(important_features)


Hours_Awake                     0.251515
Decisions_Made                  0.199418
Error_Rate                      0.160888
Task_Switches                   0.110021
Task_Completion_Time            0.084997
Cognitive_Load_Score            0.056567
Stress_Cognitive_Interaction    0.027314
Sleep_Hours_Last_Night          0.027124
Sleep_Debt                      0.025980
Sleep_Caffeine_Interaction      0.016936
Avg_Decision_Time_sec           0.016364
Stress_Level_1_10               0.011956
Caffeine_Intake_Cups            0.007695
Time_of_Day                     0.003226
dtype: float64


In [54]:
important_features = important_features[important_features > 0.01].index.to_list()

In [55]:
important_features

['Hours_Awake',
 'Decisions_Made',
 'Error_Rate',
 'Task_Switches',
 'Task_Completion_Time',
 'Cognitive_Load_Score',
 'Stress_Cognitive_Interaction',
 'Sleep_Hours_Last_Night',
 'Sleep_Debt',
 'Sleep_Caffeine_Interaction',
 'Avg_Decision_Time_sec',
 'Stress_Level_1_10']

In [60]:
X_train = X_train[important_features]
X_test = X_test[important_features]

In [58]:
import numpy as np

In [61]:
num_features = X_train.select_dtypes(include=np.number).columns

skewness = X_train[num_features].skew()

skewed_features = skewness[abs(skewness) > 0.70].index.tolist()
print("Skewed features:", skewed_features)


Skewed features: ['Error_Rate', 'Task_Completion_Time', 'Stress_Cognitive_Interaction', 'Sleep_Caffeine_Interaction', 'Stress_Level_1_10']


In [62]:

X_train_transformed = X_train.copy()
X_test_transformed = X_test.copy()

for col in skewed_features:

    if (X_train_transformed[col] >= 0).all():
        X_train_transformed[col] = np.log1p(X_train_transformed[col])
        X_test_transformed[col] = np.log1p(X_test_transformed[col])

In [64]:
out_path = r'C:\Fatigue_Level\Data\Engineered_Data'
os.makedirs(out_path, exist_ok=True)
path = os.path.join(out_path, 'X_train_transformed.csv')
X_train_transformed.to_csv(path, index=False)

In [65]:
out_path = r'C:\Fatigue_Level\Data\Engineered_Data'
os.makedirs(out_path, exist_ok=True)
path = os.path.join(out_path, 'X_test_transformed.csv')
X_test_transformed.to_csv(path, index=False)